In [1]:
import os, sys, glob, csv
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
import xlsxwriter
import random
from random import shuffle
from math import log, floor
import re
import collections
from collections import Counter
import string
import unicodedata as udata
# import pause, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from distutils.dir_util import copy_tree
import sklearn
from sklearn.metrics import *
import itertools as it
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import functools
import spacy

import tensorflow.keras.preprocessing.text as T
import torch

stop words

In [2]:
stop_words_1 = (stopwords.words('english')) #Stopword
with open('data/preprocess/stop_words.txt') as f:
    stop_words_2 = f.read().splitlines() #stop_list1
stop_words_3 = pickle.load(open('data/preprocess/stop_list2.pkl','rb'))
stop_words_all = set(stop_words_1 + stop_words_2 + stop_words_3)
len(stop_words_all) # stopwords#

594

Prepare preprocess

In [2]:
sp_tokenizer = spacy.load("en")
dil= r"[!\"#$%&()*+,\-\./:;<\=>?\@[\\]\^_`{|}~\º]+\ *" 
dil_filter = 'ºÑ!\"#$%&()*+,-./:;<=>?@[\\]^_\`{|}~\t\n'
glove = pd.read_table('./data/preprocess/glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
glove_words = set(glove.index.tolist())
len(glove_words)

2196009

以下deprecated

In [4]:
def text_preprcess(text,tokenizers='nltk'):
    '''
    Input: string
    Return: preprocessed list
    '''
    tokens = [i for i in text.split() if i not in stop_words_all]
    tokens_str = ' '.join(tokens)
    if tokenizers == 'keras':
        tokens = T.text_to_word_sequence(tokens_str,filters=dil_filter) # lower casse & tokenize
        tokens = list(filter(None, tokens))
    elif tokenizers == 'nltk':
#         tokens = re.sub(dil," ",tokens_str.lower())
        tokens = [i for i in tokens_str.lower() if i not in dil_filter]
        tokens = ''.join(tokens)
        tokens = word_tokenize(tokens)
        tokens = list(filter(None, tokens))
#         tokens = [x.lower() for x in tokens]
    elif tokenizers == 'spacy':
#         tokens = re.sub(dil," ",tokens_str.lower())
        tokens = [i for i in tokens_str.lower() if i not in dil_filter]
        tokens = ''.join(tokens)
        tokens = sp_tokenizer(tokens)
        token_li = []
        for token in tokens:
            token_li.append(token.text)
        tokens = list(filter(None, token_li))
#         tokens = [x.lower() for x in token_li]
    tokens_li = []
    for token in tokens:
        tokens_li.append(''.join([i for i in token if not i.isdigit()]))
    tokens_str = ' '.join(tokens_li)
    tokens_str = re.sub(dil," ",tokens_str.lower())
    tokens_li = tokens_str.split()
    tokens_li = list(filter(None, tokens_li))
    ori_tokens_num = len(tokens_li)
    tokens_li = [i for i in tokens_li if i in glove_words]
    try:
        rate = len(tokens_li)/ori_tokens_num
    except ZeroDivisionError:
        rate = np.nan
    return tokens_li , rate#ori_tokens_num/len(tokens_li)
        
    

In [5]:
# train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num','preprocess_text','preprocss_tokenize_num'])
train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num',
                                 'text_keras','rate_keras','text_nltk','rate_nltk','text_apacy','rate_spacy'])
train_dir = './data/20news-bydate-v3/20news-bydate-train/'
cat_dir = next(os.walk(train_dir))[1]
count = 0
for cat in tqdm(cat_dir):
    in_dir = train_dir + cat + '/'
    news_list = next(os.walk(in_dir))[2]
    news_list = [os.path.join(in_dir, f) for f in news_list]
    news_list = list(filter(lambda f: f.endswith(".txt"), news_list))
    for news_path in news_list:
        id_num = news_path.split('/')[-1].split('.')[0]
        with open(news_path,'r',encoding='latin1') as f:
            news_text = f.read()
            news_text = re.sub('\n',' ',news_text)
            news_text = re.sub('\t',' ',news_text)
            news_text_tok_ori = T.text_to_word_sequence(news_text,filters='',lower=False)
            news_text_ori = " ".join(news_text_tok_ori)
            news_text_tok_num_ori = len(news_text_tok_ori)
            news_text_tok_keras,rate_keras = text_preprcess(news_text,tokenizers='keras')
            news_text_tok_nltk,rate_nltk = text_preprcess(news_text,tokenizers='nltk')
            news_text_tok_spacy,rate_spacy = text_preprcess(news_text,tokenizers='spacy')
        train_df.loc[count] = [id_num,cat,news_text_ori,news_text_tok_num_ori,
                               news_text_tok_keras,rate_keras,news_text_tok_nltk,rate_nltk,news_text_tok_spacy,rate_spacy]
        count+=1
#         break
    
# train_df.loc[0] = [0,0,0]
print(train_df.rate_keras.mean() , train_df.rate_nltk.mean(),train_df.rate_spacy.mean()  )
train_df

 25%|██▌       | 5/20 [01:02<03:16, 13.08s/it]

KeyboardInterrupt: 

* keras tokenizer: 95.95% tokens in glove
* nltk tokenizer: 95.96% tokens in glove
* spacy tokenizer: 96.05% tokens in glove

In [ ]:
train_df = pd.DataFrame(columns=['id','category','text','original_tokenize_num','preprocess_text','preprocss_tokenize_num'])
train_dir = './data/20news-bydate-v3/20news-bydate-train/'
cat_dir = next(os.walk(train_dir))[1]
count = 0
for cat in tqdm(cat_dir):
    in_dir = train_dir + cat + '/'
    news_list = next(os.walk(in_dir))[2]
    news_list = [os.path.join(in_dir, f) for f in news_list]
    news_list = list(filter(lambda f: f.endswith(".txt"), news_list))
    for news_path in news_list:
        id_num = news_path.split('/')[-1].split('.')[0]
        with open(news_path,'r',encoding='latin1') as f:
            news_text = f.read()
            news_text = re.sub('\n',' ',news_text)
            news_text_tok_ori = T.text_to_word_sequence(news_text,filters='',lower=False)
            news_text_ori = " ".join(news_text_tok_ori)
            news_text_tok_num_ori = len(news_text_tok_ori)
#             news_text_tok_pre,rate_keras = text_preprcess(news_text,tokenizers='keras')
#             news_text_tok_pre,rate_nltk = text_preprcess(news_text,tokenizers='nltk')
            news_text_tok_pre,rate_spacy = text_preprcess(news_text,tokenizers='spacy')
            news_text_pre = " ".join(news_text_tok_pre)
        train_df.loc[count] = [id_num,cat,news_text_ori,news_text_tok_num_ori,news_text_pre,len(news_text_tok_pre)]
        count+=1
        break
    
train_df

以上deprecated

In [3]:
def text_preprcess2(text):
    '''
    Goal: filter out punctuations、words not in GloVe
    * cased
    * spacy
    * for RoBERTa + GloVe embedding
    Input: text string
    Return: preprcessed text string
    '''
    tokens = [i for i in text if i not in dil_filter] #濾掉符號
    tokens = ''.join(tokens)
    tokens = sp_tokenizer(tokens)
    token_li = []
    for token in tokens:
        token_li.append(token.text)
    tokens_li = [i for i in token_li if i in glove_words] #只留有在glove的字
    tokens = list(filter(None, tokens_li))
    tokens_str = ' '.join(tokens)
    tokens_str = re.sub(dil," ",tokens_str) #在確認濾一次符號
    return tokens_str
    

In [4]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.cuda() #gpu
# roberta.cpu()
roberta.eval() 

Using cache found in /home/leoqaz12/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/leoqaz12/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [50]:
train_df = pd.DataFrame(columns=['id','category','preprocess_text','tokens_num'])#,'preprocess_text','preprocss_tokenize_num'])
train_dir = './data/20news-bydate-v3/20news-bydate-train/'
cat_dir = next(os.walk(train_dir))[1]
count = 0
for cat in tqdm(cat_dir):
    in_dir = train_dir + cat + '/'
    news_list = next(os.walk(in_dir))[2]
    news_list = [os.path.join(in_dir, f) for f in news_list]
    news_list = list(filter(lambda f: f.endswith(".txt"), news_list))
    for news_path in news_list:
        id_num = news_path.split('/')[-1].split('.')[0]
        with open(news_path,'r',encoding='latin1') as f:
            news_text = f.read()
            news_text = re.sub('\n',' ',news_text)
            news_text = re.sub('\t',' ',news_text)
            news_text_pre = text_preprcess2(news_text)
        try:
            doc = roberta.extract_features_aligned_to_words(news_text_pre)
            tokens_num = len(doc)
        except AssertionError:
            tokens_num = np.nan
        except Exception as e:
            tokens_num_ = str(e)
            try:
                tokens_num = int(tokens_num_.split(':')[-1].split()[0])
            except IndexError:
                tokens_num = 0
                print('==ERR==',id_num,tokens_num_,'=====')
        train_df.loc[count] = [int(id_num),cat,news_text_pre,tokens_num]
        count+=1        
# GPU: 10:06
# CPU: 75
train_df

  0%|          | 0/20 [00:00<?, ?it/s]

==ERR== 18857  =====
==ERR== 19309  =====
==ERR== 19148  =====


  5%|▌         | 1/20 [00:31<09:56, 31.40s/it]

==ERR== 21139  =====
==ERR== 21056  =====
==ERR== 21194  =====
==ERR== 21159  =====


 10%|█         | 2/20 [00:59<09:05, 30.32s/it]

...

==ERR== 24750  =====
==ERR== 24546  =====
==ERR== 24744  =====


 90%|█████████ | 18/20 [09:11<01:02, 31.04s/it]

==ERR== 28128  =====
==ERR== 28005  =====


100%|██████████| 20/20 [10:04<00:00, 28.83s/it]


,id,category,preprocess_text,tokens_num
0,19305,rec.sport.hockey,My votes FWIW Team MVP Pat Verbeek He fans on ...,514
1,19313,rec.sport.hockey,Kovalev is too talented a player to play for R...,165
2,19122,rec.sport.hockey,New Jersey 1 0 23 Pittsburgh 2 3 16 First peri...,1116
3,19312,rec.sport.hockey,First of all the Penguins WILL win the cup aga...,68
4,19115,rec.sport.hockey,You ca n't But good luck trying,9
...,...,...,...,...
11309,20797,alt.atheism,When the object of their belief is said to be ...,273
11310,20537,alt.atheism,Koff You mean that as long as I put you to sle...,25
11311,20757,alt.atheism,Mr Connor 's assertion that more complex later...,131
11312,20677,alt.atheism,excess stuff deleted I know of a similar incid...,142


In [59]:
train_df[train_df['tokens_num'].isnull()] #assertion err
# train_df[train_df['id'] == 26339]

,id,category,preprocess_text,tokens_num
4948,26339,rec.sport.baseball,For your information Lankford is injured I thi...,NaN
5099,26088,rec.sport.baseball,Mr Hernandez Ñ I apologize for the misundersta...,NaN
7090,25868,talk.religion.misc,515 thus fitting in neatly with something else...,NaN
8440,26596,comp.sys.mac.hardware,Send follow ups to Sorry about the header but ...,NaN


In [60]:
train_df[train_df['tokens_num'] == 0] #no text tokens

,id,category,preprocess_text,tokens_num
150,18857,rec.sport.hockey,,0
252,19309,rec.sport.hockey,,0
318,19148,rec.sport.hockey,,0
734,21139,comp.graphics,,0
757,21056,comp.graphics,,0
853,21194,comp.graphics,,0
881,21159,comp.graphics,,0
1222,21844,talk.politics.guns,,0
1761,22811,talk.politics.misc,,0
2433,23437,talk.politics.mideast,,0


In [65]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [67]:
kk = train_df.tokens_num.tolist()
kk = [x for x in kk if str(x) != 'nan']
basic_statistics(kk)

,length
mean,190.810168
std,544.026512
mode,41.000000
min,0.000000
q1,43.000000
median,84.000000
q3,166.000000
max,12700.000000
iqr,123.000000
outlier,350.500000


In [77]:
#只留下>=3 tokens & <=512 tokens的文章
train_df_select = train_df[(train_df['tokens_num'] >=3) & (train_df['tokens_num']<=512)]
basic_statistics(train_df_select.tokens_num.tolist())

,length
mean,107.035535
std,91.436267
mode,41.000000
min,3.000000
q1,41.000000
median,78.000000
q3,144.000000
max,498.000000
iqr,103.000000
outlier,298.500000


In [48]:
roberta.extract_features_aligned_to_words(train_df[train_df['id'] == '26339'].preprocess_text.tolist()[0])

AssertionError: 

In [26]:
train_df[train_df['tokens_num'].apply(lambda x: type(x)==str)]
# train_df[type(train_df.tokens_num) is str]

,id,category,preprocess_text,tokens_num


In [9]:
type(25) is int

True

In [20]:
def convert(series):
    if type(series) is not int:
        try:
            return int(series.split(':')[-1].split()[0])
        except:
            print(series,type(series))
            return series
    return series

In [21]:
train_df['tokens_num'] = train_df.tokens_num.apply(convert)

 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>
 <class 'str'>


In [19]:
train_df.preprocess_text[0]

'My votes FWIW Team MVP Pat Verbeek He fans on 25 of goal mouth feeds but he still has 36 goals after a terrible start and has been an examplary sp team captain throughout a tough couple of seasons Honorable mention Nick Kypreos and Mark Janssens Probably more appropriate in the unsung heroes category than MVP but Kypreos 17 goals 320 PIM has been the hardest working player on the team and Janssens is underrated as a defensive center and checker I guess I place a greater emphasis on hard work than skill when determining value Biggest surprise Geoff Sanderson He had 13 goals and 31 points last season as a center then moved to left wing and has so far put up 45 goals and 80 points He now has a new Whaler record 21 power play goals most all coming from the right wing faceoff circle his garden spot Honorable mention Andrew Cassels and Terry Yake The kiddie quartet of Sanderson Poulin Nylander and Petrovicky have been attracting the most attention but Cassels is just 23 and will score close

In [17]:
'ValueError: tokens exceeds maximum length: 2550 > 512'.split(':')[-1].split()[0]

'2550'

In [64]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [147]:
tokens_num_li = train_df.preprocss_tokenize_num.tolist()
basic_statistics(tokens_num_li)

,length
mean,144.450000
std,214.881008
mode,23.000000
min,0.000000
q1,23.750000
median,63.500000
q3,157.500000
max,912.000000
iqr,133.750000
outlier,358.125000


In [45]:
news_text_tok

['just',
 'what',
 'do',
 'gay',
 'people',
 'do',
 'that',
 'straight',
 'people',
 "don't?",
 'absolutely',
 'nothing.',
 "i'm",
 'a',
 'very',
 'straight(as',
 'an',
 'arrow),',
 '17-year',
 'old',
 'male',
 'that',
 'is',
 'involved',
 'in',
 'the',
 'bsa.',
 'i',
 "don't",
 'care',
 'what',
 'gay',
 'people',
 'do',
 'among',
 'each',
 'other,',
 'as',
 'long',
 'as',
 'they',
 "don't",
 'make',
 'passes',
 'at',
 'me',
 'or',
 'anything.',
 'at',
 'my',
 'summer',
 'camp',
 'where',
 'i',
 'work,',
 'my',
 'boss',
 'is',
 'gay.',
 'not',
 'in',
 'a',
 "'pansy'",
 'way',
 'of',
 'gay',
 '(i',
 'know',
 'a',
 'few),',
 'but',
 'just',
 "'one",
 'of',
 'the',
 "guys'.",
 'he',
 "doesn't",
 'push',
 'anything',
 'on',
 'me,',
 'and',
 'we',
 'give',
 'him',
 'the',
 'same',
 'respect',
 'back,',
 'due',
 'to',
 'his',
 'position.',
 'if',
 'anything,',
 'the',
 'bsa',
 'has',
 'taught',
 'me,',
 'i',
 "don't",
 'know,',
 'tolerance',
 'or',
 'something.',
 'before',
 'i',
 'met',
 't

In [30]:
news_list = next(os.walk(in_dir))[2]
news_list = [os.path.join(in_dir, f) for f in news_list]
news_list

['./data/20news-bydate-v3/20news-bydate-train/alt.atheism/20556.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20748.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20810.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20574.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20692.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20705.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20519.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20565.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20835.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20649.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20684.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20872.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20739.txt',
 './data/20news-bydate-v3/20news-bydate-train/alt.atheism/20547.txt',
 './data/20news-byda

# TEST

In [3]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval() 

Using cache found in /home/leoqaz12/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/leoqaz12/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [10]:
tokens_ori = 'I have posted disp135.zip to alt.binaries.pictures.utilities ****** You may distribute this program freely for non-commercial use if no fee is gained. ****** There is no warranty. The author is not responsible for any damage caused by this program. Important changes since version 1,30: Fix bugs in file management system (file displaying). Improve file management system (more user-friendly). Fix bug in XPM version 3 reading. Fix bugs in TARGA reading/writng. Fix bug in GEM/IMG reading. Add support for PCX and GEM/IMG writing. Auto-skip macbinary header. (1) Introduction: This program can let you READ, WRITE and DISPLAY images with different formats. It also let you do some special effects(ROTATION, DITHERING ....) on image. Its main purpose is to let you convert image among different formts. Include simple file management system. Support \'slide show\'. There is NO LIMIT on image size. Currently this program supports 8, 15, 16, 24 bits display. If you want to use HiColor or TrueColor, you must have VESA driver. If you want to modify video driver, please read section (8). (2) Hardware Requirement: PC 386 or better. MSDOS 3,3 or higher. min amount of ram is 4M bytes(Maybe less memory will also work). (I recommend min 8M bytes for better performance). Hard disk for swapping(virtual memory). The following description is borrowed from DJGPP. Supported Wares: * Up to 128M of extended memory (expanded under VCPI) * Up to 128M of disk space used for swapping * SuperVGA 256-color mode up to 1024x768 * 80387 * XMS & VDISK memory allocation strategies * VCPI programs, such as QEMM, DESQview, and 386MAX Unsupported: * DPMI * Microsoft Windows Features: 80387 emulator, 32-bit unix-ish environment, flat memory model, SVGA graphics. (3) Installation: Video drivers, emu387 and go32.exe are borrowed from DJGPP. (If you use Western Digital VGA chips, read readme.wd) (This GO32.EXE is a modified version for vesa and is COMPLETELY compatible with original version) + *** But some people report that this go32.exe is not compatible with + other DJGPP programs in their system. If you encounter this problem, + DON\'T put go32.exe within search path. *** Please read runme.bat for how to run this program. If you choose xxxxx.grn as video driver, add \'nc 256\' to environment GO32. For example, go32=driver x:/xxxxx/xxxxx.grn nc 256 If you don\'t have 80x87, add \'emu x:/xxxxx/emu387\' to environment GO32. For example, go32=driver x:/xxxxx/xxxxx.grd emu x:/xxxxx/emu387 **** Notes: 1. I only test tr8900.grn, et4000.grn and vesa.grn. Other drivers are not tested. 2. I have modified et4000.grn to support 8, 15, 16, 24 bits display. You don\'t need to use vesa driver. If et4000.grn doesn\'t work, please try vesa.grn. 3. For those who want to use HiColor or TrueColor display, please use vesa.grn(except et4000 users). You can find vesa BIOS driver from : wuarchive.wustl.edu: /mirrors/msdos/graphics godzilla.cgl.rmit.oz.au: /kjb/MGL (4) Command Line Switch: + Usage : display [-d|--display initial_display_type] + [-s|--sort sort_method] + [-h|-?] Display type: 8(SVGA,default), 15, 16(HiColor), 24(TrueColor) + Sort method: \'name\', \'ext\' (5) Function Key: F2 : Change disk drive + CTRL-A -- CTRL-Z : change disk drive. F3 : Change filename mask (See match.doc) F4 : Change parameters F5 : Some effects on picture, eg. flip, rotate .... F7 : Make Directory t : Tag file + : Tag group files (See match.doc) T : Tag all files u : Untag file - : Untag group files (See match.doc) U : Untag all files Ins : Change display type (8,15,16,24) in \'read\' & \'screen\' menu. F6,m,M : Move file(s) F8,d,D : Delete file(s) r,R : Rename file c,C : Copy File(s) z,Z : Display first 10 bytes in Ascii, Hex and Dec modes. + f,F : Display disk free space. Page Up/Down : Move one page TAB : Change processing target. Arrow keys, Home, End, Page Up, Page Down: Scroll image. Home: Left Most. End: Right Most. Page Up: Top Most. Page Down: Bottom Most. in \'screen\' & \'effect\' menu : Left,Right arrow: Change display type(8, 15, 16, 24 bits) s,S : Slide Show. ESCAPE to terminate. ALT-X : Quit program without prompting. + ALT-A : Reread directory. Escape : Abort function and return. (6) Support Format: Read: GIF(.gif), Japan MAG(.mag), Japan PIC(.pic), Sun Raster(.ras), Jpeg(.jpg), XBM(.xbm), Utah RLE(.rle), PBM(.pbm), PGM(.pgm), PPM(.ppm), PM(.pm), PCX(.pcx), Japan MKI(.mki), Tiff(.tif), Targa(.tga), XPM(.xpm), Mac Paint(.mac), GEM/IMG(.img), IFF/ILBM(.lbm), Window BMP(.bmp), QRT ray tracing(.qrt), Mac PICT(.pct), VIS(.vis), PDS(.pds), VIKING(.vik), VICAR(.vic), FITS(.fit), Usenix FACE(.fac). the extensions in () are standard extensions. Write: GIF, Sun Raster, Jpeg, XBM, PBM, PGM, PPM, PM, Tiff, Targa, XPM, Mac Paint, Ascii, Laser Jet, IFF/ILBM, Window BMP, + Mac PICT, VIS, FITS, FACE, PCX, GEM/IMG. All Read/Write support full color(8 bits), grey scale, b/w dither, and 24 bits image, if allowed for that format. (7) Detail: Initialization: Set default display type to highest display type. Find allowable screen resolution(for .grn video driver only). 1. When you run this program, you will enter \'read\' menu. Whthin this menu you can press any function key except F5. If you move or copy files, you will enter \'write\' menu. the \'write\' menu is much like \'read\' menu, but only allow you to change directory. + The header line in \'read\' menu includes "(d:xx,f:xx,t:xx)". + d : display type. f: number of files. t: number of tagged files. pressing SPACE in \'read\' menu will let you select which format to use for reading current file. pressing RETURN in \'read\' menu will let you reading current file. This program will automatically determine which format this file is. The procedure is: First, check magic number. If fail, check standard extension. Still fail, report error. pressing s or S in \'read\' menu will do \'Slide Show\'. If delay time is 0, program will wait until you hit a key (except ESCAPE). If any error occurs, program will make a beep. ESCAPE to terminate. pressing Ins in \'read\' menu will change display type. pressing ALT-X in \'read\' menu will quit program without prompting. 2. Once image file is successfully read, you will enter \'screen\' menu. Within this menu F5 is turn on. You can do special effect on image. pressing RETURN: show image. in graphic mode, press RETURN, SPACE or ESCAPE to return to text mode. pressing TAB: change processing target. This program allows you to do special effects on 8-bit or 24-bit image. pressing Left,Right arrow: change display type. 8, 15, 16, 24 bits. pressing SPACE: save current image to file. B/W Dither: save as black/white image(1 bit). Grey Scale: save as grey image(8 bits). Full Color: save as color image(8 bits). True Color: save as 24-bit image. This program will ask you some questions if you want to write image to file. Some questions are format-dependent. Finally This program will prompt you a filename. If you want to save file under another directory other than current directory, please press SPACE. after pressing SPACE, you will enter \'write2\' menu. You can change directory to what you want. Then, pressing SPACE: this program will prompt you \'original\' filename. pressing RETURN: this program will prompt you \'selected\' filename (filename under bar). 3. This program supports 8, 15, 16, 24 bits display. 4. This Program is MEMORY GREEDY. If you don\'t have enough memory, the performance is poor. 5. If you want to save 8 bits image : try GIF then TIFF(LZW) then TARGA then Sun Raster then BMP then ... If you want to save 24 bits image (lossless): try TIFF(LZW) or TARGA or ILBM or Sun Raster (No one is better for true 24bits image) 6. I recommend Jpeg for storing 24 bits images, even 8 bits images. 7. Not all subroutines are fully tested 8. This document is not well written. If you have any PROBLEM, SUGGESTION, COMMENT about this program, Please send to u7711501@bicmos.ee.nctu.edu.tw (140,113,11,13). I need your suggestion to improve this program. (There is NO anonymous ftp on this site) (8) Tech. information: Program (user interface and some subroutines) written by Jih-Shin Ho. Some subroutines are borrowed from XV(2,21) and PBMPLUS(dec 91). Tiff(V3,2) and Jpeg(V4) reading/writing are through public domain libraries. Compiled with DJGPP. You can get whole DJGPP package from SIMTEL20 or mirror sites. For example, wuarchive.wustl.edu: /mirrors/msdos/djgpp (9) For Thoese who want to modify video driver: 1. get GRX source code from SIMTEL20 or mirror sites. 2. For HiColor and TrueColor: 15 bits : # of colors is set to 32768. 16 bits : # of colors is set to 0xc010. 24 bits : # of colors is set to 0xc018. Acknowledgment: I would like to thank the authors of XV and PBMPLUS for their permission to let me use their subroutines. Also I will thank the authors who write Tiff and Jpeg libraries. Thank DJ. Without DJGPP I can\'t do any thing on PC.'

In [12]:
# tokens_ori = train_df.loc[3,'preprocess_text']#'how\'s are you'#
#tokens = roberta.encode(tokens_ori)
# assert tokens.tolist() == [0, 31414, 232, 328, 2]
# assert roberta.decode(tokens) == 'Hello world!'
#print(len(tokens) , tokens)
#roberta.decode(tokens)
doc = roberta.extract_features_aligned_to_words(tokens_ori)
print(len(doc))
for tok in doc:
    print('{:10}{} (...)'.format(str(tok), tok.vector[:5]))

ValueError: tokens exceeds maximum length: 2550 > 512

In [189]:
tokens = 'I said, "hello RoBERTa."'
tokens = sp_tokenizer(tokens)
token_li = []
for token in tokens:
    token_li.append(token.text)
len(token_li) , token_li

(8, ['I', 'said', ',', '"', 'hello', 'RoBERTa', '.', '"'])

In [167]:
last_layer_features = roberta.extract_features(tokens)
print(last_layer_features.size())# == torch.Size([1, 5, 1024])

all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
assert len(all_layers) == 25
assert torch.all(all_layers[-1] == last_layer_features)

ValueError: tokens exceeds maximum length: 592 > 512

In [163]:
last_layer_features.detach().numpy().shape

(1, 19, 1024)

In [168]:
from pytorch_transformers import RobertaTokenizer

In [173]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')

100%|██████████| 456318/456318 [00:00<00:00, 545019.70B/s]


In [178]:
len(tokenizer.encode(train_df.loc[0,'text']))

W0925 15:40:29.729804 139833327294272 tokenization_utils.py:665] Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


590